## BPE Tokenizer

BPE(Byte-Pair Encoding) Tokenizer 是基于统计规律构建子词（sub-word）分词器，被 GPT-1 采用，并成为主流的大模型分词方法。

分词单位常见的有：

- 字符：'a','b','!','_','好', 具有基础的字符词表，可以表示任意文本，确定是编码后的序列太长，对于 序列模型 来说编码效率低影响计算效率。
- 单词：'hello ', 'world','!'，其形式严格，扩展性差。
- 子词：是一种囊括字符、单词和连续文本的一种文本定义，其形式自由灵活，且对通用语种有较强的适配性。例如：'hello', 'he', 'h', 'hello world!', '!', 'ld', ... 都可以视为子词的范畴。其包含基础的字符词表，可以继承编码任意文本特性，同时，其序列长度更短，较单词又更灵活，且通用性更高。

子词类分词器的缺陷是什么？

- 数字：'9.11' 和 '9.8' 谁更大？ 
- 字符：'strawberry' 由几个 'r'？

分词器由两个组件构成：

1. 规则：规定了序列的划分方式，其划分规则具有唯一性，固定规则下产生唯一词表、编码和解码结果
2. 词表：根据规则在语料上进行切分序列得到的子词

分词器使用包含三个过程，其各阶段的输入输出分别是什么？

1. 训练
2. 编码
3. 解码

回到 BPE, Byte 指的是文本在计算机中的最小表示单位，即字符。Pair 指的是相邻 Byte 之间可以进行**融合**形成 **新的Byte**（新的 Byte 也当成是最小文本单位）。在 BPE 中，关键其实是 Pair 按照什么规则来构建，这是 BPE 算法的核心。

## Byte

In [1]:
print('bytes:', bytes([97]))
print('a'.encode("utf-8"))
print('b'.encode("utf-8"))
print('1'.encode("utf-8"))
print(b'\x01'.decode())
print(b'\x02'.decode())
print(b'\x09'.decode())
print(b'\x7f'.decode())

bytes: b'a'
b'a'
b'b'
b'1'


	



一段文本可以用 UTF-8 转化成 Byte 序列

In [2]:
text = '''   
Large Language Models is all you need,
what can i say, manba out. 
Attention is All you need.
Vision Transformers, 
Generative Pretrained Transformers,
Reinforcement leraning from human feedback
chain of thought is basic reasoning tool.
LLMs can evaluate NLP results.
Richard Sutton Refinforcement Learning Introduction edition 2.
encoder-only framework
'''

text_bytes = text.encode("utf-8") # raw bytes
ids = list(text_bytes) # list of integers in range 0..255
print(len(ids))
for i in range(10):
    print(f'byte pisition_{i} : {ids[i]}:\t',text_bytes[:i])

358
byte pisition_0 : 32:	 b''
byte pisition_1 : 32:	 b' '
byte pisition_2 : 32:	 b'  '
byte pisition_3 : 10:	 b'   '
byte pisition_4 : 76:	 b'   \n'
byte pisition_5 : 97:	 b'   \nL'
byte pisition_6 : 114:	 b'   \nLa'
byte pisition_7 : 103:	 b'   \nLar'
byte pisition_8 : 101:	 b'   \nLarg'
byte pisition_9 : 32:	 b'   \nLarge'


## 中文Bytes

In [3]:
cn_text = "小"
print(cn_text.encode("utf-8"))
print(b'\xe5\xb0\x8f'.decode())
# print(b'\xe5\xb0'.decode()) # 报错
# print(b'\xe5'.decode()) # 报错

cn_text = "冬"
print(cn_text.encode("utf-8"))

cn_text = "小冬瓜"
print(cn_text.encode("utf-8"))

print(cn_text.encode("utf-8").decode())

b'\xe5\xb0\x8f'
小
b'\xe5\x86\xac'
b'\xe5\xb0\x8f\xe5\x86\xac\xe7\x93\x9c'
小冬瓜


## 初始化 byte 词表

- 词表是字典（或者是map），更具体的数据结构是双向的字典（map）
- 初始化字典具有 常见的英文字母和标点符号。
- BPE 原始论文或实现仅应用于英文场景，更加通用的实现是将语料里按照最基础的 bytes 切分求 set
- 举例，如果一个字 “冬” 在初始化词表里未出现，那么 BPE 训练或编码就无法实现 merge

In [4]:
def build_vocab(size = 256):
    merges = {}
    vocab = {idx: bytes([idx]) for idx in range(size)}
    for (p0, p1), idx in merges.items():
        vocab[idx] = vocab[p0] + vocab[p1]
    return vocab

vocab = build_vocab(size = 256)
print(vocab) # 字典

{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87: b'W', 88: b'X', 89: b'Y', 90: b'Z', 91: b'[',

## Byte-Pair

Byte Pair 包含两个步骤：

1. 构建 pair
2. 统计 pair
3. 选择 pair: 选择出现频次最高的 相邻 byte-pair 。

In [5]:
# 构建 pair
example = [1, 2, 3, 1, 2] 

print(example[:-1],'\n',
      example[1:])

[1, 2, 3, 1] 
 [2, 3, 1, 2]


In [6]:
# 构建-统计 pair
def get_stats(ids, counts=None):
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]): # iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts
print('get stats')
example = [1, 2, 3, 1, 2] # token id 序列
counts = get_stats(example)
print(counts) # 相邻token出现频次

pair = max(counts, key=counts.get) 
print('最大对:', pair)

get stats
{(1, 2): 2, (2, 3): 1, (3, 1): 1}
最大对: (1, 2)


## Merge Pair

Merge Pair 是 BPE 的分词规则。

- 筛选最大频次相邻 byte-pair，合并成新的byte
- 更新序列

输入：语料

输出：

- 新的byte: 'new_byte', 扩充 byte 词表
- 新的序列: 'newids', 达到压缩序列长度目的

In [7]:
def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2 # 相邻两个token id 匹配上Pair, 那么就进行替换
        else:
            newids.append(ids[i])
            i += 1
    return newids
ids=[1, 2, 3, 1, 2]
pair=(1, 2)
new_byte = max(ids) + 1
newids = merge(ids, pair, new_byte) 
print(newids)

[4, 3, 4]


## BPE Train

BPE 训练则反复的 执行 Merge Pair 操作，产生最终 词表和序列。

In [8]:
INITIAL_VOCAB_SIZE = 256

class BasicTokenizer():
    def __init__(self):
        # def __init__(self):
        self.merges = {} # (int, int) -> int
        self.vocab = self.build_vocab() # int -> bytes
        
    def build_vocab(self):
        vocab = {idx: bytes([idx]) for idx in range(INITIAL_VOCAB_SIZE)}
        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]
        return vocab

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= INITIAL_VOCAB_SIZE
        num_merges = vocab_size - INITIAL_VOCAB_SIZE

        text_bytes = text.encode("utf-8") 
        ids = list(text_bytes) 

        merges = {} 
        # int -> bytes
        vocab = {idx: bytes([idx]) for idx in range(INITIAL_VOCAB_SIZE)} 
        for i in range(num_merges):
            pre_ids = ids
            stats = get_stats(ids)
            # pair(2,3),    vocab[2]='te', vocab[3]='st'
            pair = max(stats, key=stats.get)    
            idx = 256 + i
            ids = merge(ids, pair, idx)
            merges[pair] = idx
            
            # 原来的词不会剔除，而是在基础词表上累加，如
            # vocab[new_id] = 'te' + 'st' -> vocab[4] = 'test'
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]] 
            
            if verbose:
                # print(f'loop {i}', pre_ids, '->',)
                print(f'loop {i}', [ vocab[tmp_idx].decode() for tmp_idx in pre_ids])
                print(f'loop {i}', 'max pair', pair, vocab[pair[0]].decode(), vocab[pair[1]].decode() , '-> new byte:', idx, vocab[idx].decode())
                # print(f'loop {i}', ids)
                print(f'loop {i}', [ vocab[tmp_idx].decode() for tmp_idx in ids])
                print('-'*20)
        self.merges = merges # used in encode()
        self.vocab = vocab   # used in decode()

bpe = BasicTokenizer()


dummy_text = 'abcababcaabc'
new_byte_nums = 4
bpe.train(dummy_text, vocab_size = INITIAL_VOCAB_SIZE+new_byte_nums, verbose = True) # 
print('基础词表:',bpe.vocab[97], bpe.vocab[98], bpe.vocab[99])
print('合并的 byte-pair:',bpe.merges)
for i in range(256, INITIAL_VOCAB_SIZE+new_byte_nums,):
    print(i, bpe.vocab[i])

loop 0 ['a', 'b', 'c', 'a', 'b', 'a', 'b', 'c', 'a', 'a', 'b', 'c']
loop 0 max pair (97, 98) a b -> new byte: 256 ab
loop 0 ['ab', 'c', 'ab', 'ab', 'c', 'a', 'ab', 'c']
--------------------
loop 1 ['ab', 'c', 'ab', 'ab', 'c', 'a', 'ab', 'c']
loop 1 max pair (256, 99) ab c -> new byte: 257 abc
loop 1 ['abc', 'ab', 'abc', 'a', 'abc']
--------------------
loop 2 ['abc', 'ab', 'abc', 'a', 'abc']
loop 2 max pair (257, 256) abc ab -> new byte: 258 abcab
loop 2 ['abcab', 'abc', 'a', 'abc']
--------------------
loop 3 ['abcab', 'abc', 'a', 'abc']
loop 3 max pair (258, 257) abcab abc -> new byte: 259 abcababc
loop 3 ['abcababc', 'a', 'abc']
--------------------
基础词表: b'a' b'b' b'c'
合并的 byte-pair: {(97, 98): 256, (256, 99): 257, (257, 256): 258, (258, 257): 259}
256 b'ab'
257 b'abc'
258 b'abcab'
259 b'abcababc'


In [9]:
# 注意空格会参与 byte-pair, 空格本身也是一个 byte
text = 'old older fast faster fastest best better yes desk mess'
bpe.train(text, vocab_size = 256+7, verbose = False )
for i in range(256, 256+7, 1):
    print(f'{i}: \'{bpe.vocab[i].decode()}\'')
print(bpe.merges)

256: 'st'
257: 'er'
258: 'er '
259: 'fa'
260: 'fast'
261: 'es'
262: 'ol'
{(115, 116): 256, (101, 114): 257, (257, 32): 258, (102, 97): 259, (259, 256): 260, (101, 115): 261, (111, 108): 262}


In [10]:
# text = 'old older fast faster fastest best better yes desk mess'
bpe.train(text, vocab_size = 270, verbose = False ) # 270-256 = 14 个新 byte
for i in range(256, 270, 1):
    print(f'{i}: \'{bpe.vocab[i].decode()}\'')
print(bpe.merges)

256: 'st'
257: 'er'
258: 'er '
259: 'fa'
260: 'fast'
261: 'es'
262: 'ol'
263: 'old'
264: 'er fast'
265: 'est'
266: 'est '
267: 'est b'
268: 'old '
269: 'old old'
{(115, 116): 256, (101, 114): 257, (257, 32): 258, (102, 97): 259, (259, 256): 260, (101, 115): 261, (111, 108): 262, (262, 100): 263, (258, 260): 264, (101, 256): 265, (265, 32): 266, (266, 98): 267, (263, 32): 268, (268, 263): 269}


由于示例文本较短，当预设目标合并词表数量较大时，263～279 byte序列会一直累增。

用于 BPE 训练的语料模式丰富且多样化，目标合并词表数量没有那么敏感。

In [11]:
chinese_text = '跟着小冬瓜AIGC一起学习LLM'

cn_bpe = BasicTokenizer()
cn_bpe.train(chinese_text, vocab_size = 270, verbose = False ) # 270-256 = 14 个新 byte
for i in range(256, 270, 1):
    # print(f'{i}: \'{cn_bpe.vocab[i].decode()}\'')
    print(f'{i}: \'{cn_bpe.vocab[i]}\'')
print(bpe.merges)

256: 'b'\xe8\xb7''
257: 'b'\xe8\xb7\x9f''
258: 'b'\xe8\xb7\x9f\xe7''
259: 'b'\xe8\xb7\x9f\xe7\x9d''
260: 'b'\xe8\xb7\x9f\xe7\x9d\x80''
261: 'b'\xe8\xb7\x9f\xe7\x9d\x80\xe5''
262: 'b'\xe8\xb7\x9f\xe7\x9d\x80\xe5\xb0''
263: 'b'\xe8\xb7\x9f\xe7\x9d\x80\xe5\xb0\x8f''
264: 'b'\xe8\xb7\x9f\xe7\x9d\x80\xe5\xb0\x8f\xe5''
265: 'b'\xe8\xb7\x9f\xe7\x9d\x80\xe5\xb0\x8f\xe5\x86''
266: 'b'\xe8\xb7\x9f\xe7\x9d\x80\xe5\xb0\x8f\xe5\x86\xac''
267: 'b'\xe8\xb7\x9f\xe7\x9d\x80\xe5\xb0\x8f\xe5\x86\xac\xe7''
268: 'b'\xe8\xb7\x9f\xe7\x9d\x80\xe5\xb0\x8f\xe5\x86\xac\xe7\x93''
269: 'b'\xe8\xb7\x9f\xe7\x9d\x80\xe5\xb0\x8f\xe5\x86\xac\xe7\x93\x9c''
{(115, 116): 256, (101, 114): 257, (257, 32): 258, (102, 97): 259, (259, 256): 260, (101, 115): 261, (111, 108): 262, (262, 100): 263, (258, 260): 264, (101, 256): 265, (265, 32): 266, (266, 98): 267, (263, 32): 268, (268, 263): 269}


## Encode

编码：指文本经过分词处理成离散token，再经词表映射到 token id 序列。

在上述词表里，请问对于一个新的 文本 `oldest`, 从单词视角，`oldest`在训练语料中未出现， 但从 byte 视角，现有词表能表示出完整文本。

那么编码结果是否为：`old` `est` 即 `[263, 265]`?

```
256: 'st'
257: 'er'
258: 'er '
259: 'fa'
260: 'fast'
261: 'es'
262: 'ol'
263: 'old'
264: 'er fast'
265: 'est'
266: 'est '
267: 'est b'
268: 'old '
269: 'old old'
```

编码如果依靠“子串匹配”，编码效率非常低。一般而言，编码与训练的分词规则是一样的。

编码类似训练。先将文本转化成最小粒度的 byte， 依次合并。 `oldest` 例子所遇到的问题在于：

 
`ol`,`ld`,`de`,`es`,`st`, 这些 byte-pair 选择哪对来进行 merge？

已知 256: `st`, 261: `es`, 262: `ol`。

In [12]:
# encoder
# utf-8 token ids
text = 'oldest'
text_bytes = text.encode("utf-8") 

# bpe token ids
ids = list(text_bytes) 
i = 0
while len(ids) >= 2:
    stats = get_stats(ids)

    # 关键代码
    # 随后 block 讲解该行代码实现
    pair = min(stats, key=lambda p: bpe.merges.get(p, float("inf")))
    
    print(pair)
    print(bpe.vocab[pair[0]].decode(), bpe.vocab[pair[1]].decode())
    print(f'loop {i}', [ bpe.vocab[tmp_idx].decode() for tmp_idx in ids])
    if pair not in bpe.merges:
        break 
    idx = bpe.merges[pair] 
    ids = merge(ids, pair, idx) 
    print(f'loop {i}', [ bpe.vocab[tmp_idx].decode() for tmp_idx in ids])
    print('-'*20)
    i+=1
print(ids)

(115, 116)
s t
loop 0 ['o', 'l', 'd', 'e', 's', 't']
loop 0 ['o', 'l', 'd', 'e', 'st']
--------------------
(111, 108)
o l
loop 1 ['o', 'l', 'd', 'e', 'st']
loop 1 ['ol', 'd', 'e', 'st']
--------------------
(262, 100)
ol d
loop 2 ['ol', 'd', 'e', 'st']
loop 2 ['old', 'e', 'st']
--------------------
(101, 256)
e st
loop 3 ['old', 'e', 'st']
loop 3 ['old', 'est']
--------------------
(263, 265)
old est
loop 4 ['old', 'est']
[263, 265]


### encode 合并顺序 trick

In [13]:
people = [
    {'name': 'Alice', 'age': 30},
    {'name': 'Bob', 'age': 25},
    {'name': 'Charlie', 'age': 35}
]
result = min(people, key=lambda person: person['age']) # 将 age 作为 key, 选择最小 age 值, 对应的 字典 item
print(result)

{'name': 'Bob', 'age': 25}


In [14]:
test_ids = [1,2,3,4,1]
stats = get_stats(test_ids)
print(stats)
merges = {}
merges[(3,4)] = 5
merges[(1,2)] = 6

pair = min(stats, key=lambda p: merges.get(p, float("inf"))) 
print(pair)

{(1, 2): 1, (2, 3): 1, (3, 4): 1, (4, 1): 1}
(3, 4)


为什么是min?  encode 的编码规则核心在于

1. 合并是从最基础的 byte 开始 merge, 而非一步到位匹配词表里的子词
2. 相邻对是否可以合并，需要参考 `bpe.merges` 表。其记录训练过程中，新 byte 的 pair 来源。所以 BPE 词表，除了存储 vocab， 还要存储 merges 表。
3. merge 时会遇到多个 byte-pair 都在 `bpe.merges` 出现， 准则是 **合并后的 byte id 越小、说明训练时出现的频次越高**。故代码实现取min

## Decode

In [15]:
print(ids)

[263, 265]


In [16]:
text_bytes = b"".join(bpe.vocab[idx] for idx in ids)
decode_text = text_bytes.decode("utf-8", errors="replace")
print(decode_text)

oldest


## 中英 Tokenizer

In [17]:
corpus_text = '''
Large Language Models is all you need,
what can i say, manba out. 
Attention is All you need.
Vision Transformers, 
Generative Pretrained Transformers,
Reinforcement leraning from human feedback
chain of thought is basic reasoning tool.
LLMs can evaluate NLP results.
Richard Sutton Refinforcement Learning Introduction edition 2.
encoder-only framework
大型语言模型就是所需的一切，
我还能说什么呢，曼巴退场。
注意力机制就是所需的一切。
视觉变换器，
生成式预训练变换器，
基于人类反馈的强化学习，
思维链是基本推理工具。
大型语言模型能评估自然语言处理结果。
理查德·萨顿《强化学习导论》第二版；
仅编码器框架。
'''

In [18]:
from collections import Counter
import copy


class BPETokenizer():
    def __init__(self, text):
        self.merges = {} # (int, int) -> int
        self.vocab = self.build_vocab(text) # int -> bytes
        
    def build_vocab(self, text):
        char_counter = Counter(text)
        sorted_chars = sorted(char_counter.items(), key=lambda x: x[1], reverse=True)
        i = 0
        vocab = {}
        for k, v in sorted_chars:
            vocab[i] = k 
            i = i + 1
        return vocab

    def train(self, text, vocab_size, verbose=False):
        num_merges = vocab_size - len(self.vocab)
        reverse_vocab = {}
        for k, v in self.vocab.items():
            reverse_vocab[v] = k
        ids = list(text) 
        ids = [ reverse_vocab[idx] for idx in ids]
        merges = {} 
        vocab = copy.deepcopy(self.vocab)
        cur_vocab_size = len(self.vocab)
        
        for i in range(num_merges):
            pre_ids = ids
            stats = get_stats(ids)
            pair = max(stats, key=stats.get)  
            idx = cur_vocab_size + i
            ids = merge(ids, pair, idx)
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]] 
            
        self.merges = merges 
        self.vocab = vocab   
        self.reverse_vocab = reverse_vocab

    def encode(self, text):
        ids = list(text) 
        ids = [ self.reverse_vocab[idx] for idx in ids]
        i = 0
        while len(ids) >= 2:
            stats = get_stats(ids)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            if pair not in self.merges:
                break 
            idx = self.merges[pair] 
            ids = merge(ids, pair, idx) 
            i+=1
        return ids

    def decode(self, ids):
        text = "".join(bpe.vocab[idx] for idx in ids)
        return text


bpe = BPETokenizer( corpus_text  )
print(len(bpe.vocab))

bpe.train(corpus_text, vocab_size = 150, verbose = True) 

input_ids = bpe.encode('reasoning')
print(input_ids)

decode_text = bpe.decode(input_ids)
print(decode_text)

122
[147, 3, 9, 4, 2, 148]
reasoning


## BPE 总结

1. BPE 是基于统计的规则分词器，分词结果（token）在统计视角是合理的，哪怕分词并不符合人类日常习惯
2. 人类主观的分词是有不一致性的
3. 分词需要考虑 文本编码后的压缩率
4. 词表越大，llm model 的 embedding、lmhead层，参数量随之增大
5. 分词器扩展实现：1.词表合并 2.手动增加token 3.增加分词器规则（如标点符号不允许被合并）